# Laden die Brustkrebsdaten in die sqlite-Databenk

In [ ]:
#Laden Libraries
import sqlite3
import pandas as pd

In [318]:
#Laden Libraries
import sqlite3
import pandas as pd

In [319]:
# Verbindung zur Datenbank
conn = sqlite3.connect('BreastCancerDB.db')
cursor = conn.cursor()
print("Datenbank wurde erfolgreich geöffnet");

Datenbank wurde erfolgreich geöffnet


In [320]:
# Löschen Tabellen wenn sie schon exestieren
cursor.execute("DROP TABLE IF EXISTS patient")
cursor.execute("DROP TABLE IF EXISTS condition")
cursor.execute("DROP TABLE IF EXISTS medication")
cursor.execute("DROP TABLE IF EXISTS observation")
cursor.execute("DROP TABLE IF EXISTS procedures")
cursor.execute("DROP TABLE IF EXISTS patient_careplan")
cursor.execute("DROP TABLE IF EXISTS patient_new")

In [321]:
# Laden Daten
patient = pd.read_csv('BreastCancerData/patients.csv', sep=",")
condition = pd.read_csv('BreastCancerData/conditions.csv', sep=",")
medication = pd.read_csv('BreastCancerData/medications.csv', sep=",")
observation = pd.read_csv('BreastCancerData/observations.csv', sep=",")
procedure = pd.read_csv('BreastCancerData/procedures.csv', sep=",")

In [322]:
# Tabellen erstellen
cursor.execute('''
		CREATE TABLE IF NOT EXISTS patient (
			Id nvarchar(36) primary key,
            BIRTHDATE Date,
            DEATHDATE Date,
            SSN nvarchar(50),
            DRIVERS nvarchar(50),
            PASSPORT nvarchar(10),
            PREFIX nvarchar(3),
            FIRST nvarchar(50),
            LAST nvarchar(50),
            SUFFIX nvarchar(50),
            MAIDEN nvarchar(50),
            MARITAL nvarchar(1),
            RACE nvarchar(10),
            ETHNICITY nvarchar(20),
            GENDER nvarchar(2),
            BIRTHPLACE nvarchar(50),
            ADDRESS nvarchar(50),
            CITY nvarchar(50),
            STATE nvarchar(50),
            COUNTY nvarchar(50),
            ZIP nvarchar(4),
            LAT nvarchar(10),
            LON nvarchar(10),
            HEALTHCARE_EXPENSES float,
            HEALTHCARE_COVERAGE float
          
			)
             ''')

cursor.execute('''
		CREATE TABLE IF NOT EXISTS condition (
            START Date,
            STOP Date,
            PATIENT nvarchar(36),
            ENCOUNTER nvarchar(50),
            DESCRIPTION object,
            CODE int,
            foreign key(PATIENT) references patient(Id) 
			)
               ''')

cursor.execute('''
		CREATE TABLE IF NOT EXISTS medication (
            START nvarchar(20),
            STOP nvarchar(20),
            PATIENT nvarchar(36),
            PAYER nvarchar(50),
            ENCOUNTER nvarchar(50),
            CODE int,
            DESCRIPTION nvarchar(50),
            BASE_COST float,
            PAYER_COVERAGE float,
            DISPENSES int,
            TOTALCOST float,
            REASONCODE float,
            REASONDESCRIPTION nvarchar(50),
            foreign key(PATIENT) references patient(Id) 
			)
               ''')

cursor.execute('''
		CREATE TABLE IF NOT EXISTS observation (
            DATE nvarchar(20),
            PATIENT nvarchar(36),
            ENCOUNTER nvarchar(50),
            CODE int,
            DESCRIPTION nvarchar(50),
            VALUE float,
            UNITS nvarchar(10),
            TYPE nvarchar(10),
            foreign key(PATIENT) references patient(Id) 
			)
               ''')

cursor.execute('''
		CREATE TABLE IF NOT EXISTS procedure (
            DATE nvarchar(20),
            PATIENT nvarchar(36),
            ENCOUNTER nvarchar(50),
            CODE int,
            DESCRIPTION nvarchar(50),
            BASE_COST float,
            REASONCODE float,
            REASONDESCRIPTION nvarchar(50),
            foreign key(PATIENT) references patient(Id) 
			)
               ''')


In [323]:
# Einfügen der Daten in die erstellten Tabellen
patient.to_sql('patient', conn, if_exists='append', index=False)
condition.to_sql('condition', conn, if_exists='append', index=False)
medication.to_sql('medication', conn, if_exists='append', index=False)
observation.to_sql('observation', conn, if_exists='append', index=False)
procedure.to_sql('procedure', conn, if_exists='append', index=False)

# Sternschema erstellen

Wir benötigen eine Faktentabelle für unsere Sternschema. Die Tabellen Patien und Condition werden als Dimensionstabellen genommen.

In [324]:
# Faktentabelle in 
cursor.execute('''
		CREATE TABLE IF NOT EXISTS patient_careplan (
            CAREPLANT_ID INTEGER PRIMARY KEY AUTOINCREMENT,
			PATIENT_ID nvarchar(36),
            CARE_TYPE nvarchar(20),
            DESCRIPTION nvarchar(50),
            RESON nvarchar(50),
            ENCOUNTER nvarchar(50),
            DATE Date,
            foreign key(PATIENT_ID) references patient(Id) 
			)
             ''')

In [325]:
cursor.execute('''
    INSERT INTO patient_careplan(PATIENT_ID,CARE_TYPE, DESCRIPTION,RESON,ENCOUNTER,DATE)
    SELECT patient.Id as PATIENT_ID, 'Medication' as CARE_TYPE, medication.DESCRIPTION, medication.REASONCODE as RESON, medication.ENCOUNTER, medication.START as DATE
    FROM patient 
    INNER JOIN 
    medication ON patient.Id = medication.PATIENT
    
    UNION
    SELECT patient.Id as PATIENT_ID,'Observation' as CARE_TYPE, observation.DESCRIPTION, observation.CODE as RESON, observation.ENCOUNTER, observation.DATE
    FROM patient 
    INNER JOIN 
    observation ON patient.Id = observation.PATIENT
    
    UNION
    SELECT patient.Id as PATIENT_ID, 'Procedure' as CARE_TYPE, procedure.DESCRIPTION, procedure.REASONDESCRIPTION as RESON, procedure.ENCOUNTER, procedure.DATE
    FROM patient 
    INNER JOIN 
    procedure ON patient.Id = procedure.PATIENT
    ''')

In [326]:
print(pd.read_sql_query('''
     SELECT *
     FROM patient_careplan
     ''', conn))

       CAREPLANT_ID                            PATIENT_ID    CARE_TYPE  \
0                 1  009121bf-a672-8942-443e-85e18a33f766  Observation   
1                 2  009121bf-a672-8942-443e-85e18a33f766  Observation   
2                 3  009121bf-a672-8942-443e-85e18a33f766  Observation   
3                 4  009121bf-a672-8942-443e-85e18a33f766  Observation   
4                 5  009121bf-a672-8942-443e-85e18a33f766  Observation   
...             ...                                   ...          ...   
32718         32719  fffc77ee-b38c-41bd-2b07-ebaca114a78e  Observation   
32719         32720  fffc77ee-b38c-41bd-2b07-ebaca114a78e  Observation   
32720         32721  fffc77ee-b38c-41bd-2b07-ebaca114a78e  Observation   
32721         32722  fffc77ee-b38c-41bd-2b07-ebaca114a78e  Observation   
32722         32723  fffc77ee-b38c-41bd-2b07-ebaca114a78e  Observation   

      DESCRIPTION RESON ENCOUNTER                  DATE  
0            DALY  DALY      None  2010-05-03T23:15:2

In [327]:
condition['DESCRIPTION'].unique()

array(['Coronary Heart Disease', 'Atrial Fibrillation', 'Cardiac Arrest',
       'History of cardiac arrest (situation)', 'Stroke',
       'Myocardial Infarction',
       'History of myocardial infarction (situation)',
       'Malignant neoplasm of breast (disorder)'], dtype=object)

In [328]:
conn.close()